# Ex
### Exercise 8.1
- 2, 6, 19, 20, 25

### Exercise 8.2
- 4, 6, 13, 14, 15, 16, 17

### Exercise 8.3
- 3, 6, 7

### Exercise 8.4
- 3, 6, 7, 8, 9

# Com Ex
### Computer Exercise 8.1
- 2, 6, 7

### Computer Exercise 8.2
- 1, 12(a), 12(b)

### Computer Exercise 8.3
- 1(a), 7

### Computer Exercise 8.4
- 3

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [16]:
# 8.1 - 2

def dolittle_decomposition(A):
    n = A.shape[0]
    U = A.copy()
    L = np.zeros((n, n))
    
    for i in range(n):
        L[i, i] = 1
        for j in range(i + 1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, i:] -= U[i, i:] * L[j, i]

    return L, U

def crout_decomposition(A):
    n = A.shape[0]
    L, U = dolittle_decomposition(A)
    LT = np.transpose(L)
    D = np.diag(np.diag(U))
    
    return L, D, LT

def verify_dolittle(A):
    L, U = dolittle_decomposition(A)
    
    print("L:\n", L)
    print("U:\n", U)
    print("LU:\n", L @ U)
    print("A:\n", A)
    if np.allclose(L @ U, A):
        print("pass")
    else:
        print("fail")
        
def verify_crout(A):
    L, D, LT = crout_decomposition(A)
    
    print("L:\n", L)
    print("D:\n", D)
    print("LT:\n", LT)
    print("LDT:\n", L @ D @ LT)
    print("A:\n", A)
    if np.allclose(L @ D @ LT, A):
        print("pass")
    else:
        print("fail")

A = np.array(
    [[5, 7, 6, 5], 
     [7, 10, 8, 7], 
     [6, 8, 10, 9], 
     [5, 7, 9, 10]]
    , dtype=float
)

verify_dolittle(A)
print()
verify_crout(A)

L:
 [[ 1.   0.   0.   0. ]
 [ 1.4  1.   0.   0. ]
 [ 1.2 -2.   1.   0. ]
 [ 1.   0.   1.5  1. ]]
U:
 [[ 5.   7.   6.   5. ]
 [ 0.   0.2 -0.4  0. ]
 [ 0.   0.   2.   3. ]
 [ 0.   0.   0.   0.5]]
LU:
 [[ 5.  7.  6.  5.]
 [ 7. 10.  8.  7.]
 [ 6.  8. 10.  9.]
 [ 5.  7.  9. 10.]]
A:
 [[ 5.  7.  6.  5.]
 [ 7. 10.  8.  7.]
 [ 6.  8. 10.  9.]
 [ 5.  7.  9. 10.]]
pass

L:
 [[ 1.   0.   0.   0. ]
 [ 1.4  1.   0.   0. ]
 [ 1.2 -2.   1.   0. ]
 [ 1.   0.   1.5  1. ]]
D:
 [[5.  0.  0.  0. ]
 [0.  0.2 0.  0. ]
 [0.  0.  2.  0. ]
 [0.  0.  0.  0.5]]
LT:
 [[ 1.   1.4  1.2  1. ]
 [ 0.   1.  -2.   0. ]
 [ 0.   0.   1.   1.5]
 [ 0.   0.   0.   1. ]]
LDT:
 [[ 5.  7.  6.  5.]
 [ 7. 10.  8.  7.]
 [ 6.  8. 10.  9.]
 [ 5.  7.  9. 10.]]
A:
 [[ 5.  7.  6.  5.]
 [ 7. 10.  8.  7.]
 [ 6.  8. 10.  9.]
 [ 5.  7.  9. 10.]]
pass


In [36]:
# 8.1 - 6, 7

def build_tridiagonal(n, a, b, c):
    A = np.diag([a] * (n - 1), -1) + np.diag([b] * n) + np.diag([c] * (n - 1), 1)
    return A

def inverse_tridiagonal_naive(A):
    L = np.zeros(A.shape)
    U = np.zeros(A.shape) 
    n = A.shape[0]
    
    L[0, 0] = 1
    U[0, :] = A[0, :]
    for i in range(1, n):
        L[i, i] = 1
        L[i, i - 1] = A[i, i - 1] / U[i - 1, i - 1]
        U[i, i] = A[i, i] - U[i - 1, i] * L[i, i - 1]
        if i < n - 1:
            U[i, i + 1] = A[i, i + 1]
        
    L_inv = np.eye(n, dtype=float)
    for i in range(1, n):
        for j in range(0, i):
            L_inv[i, j] = - L[i, i-1] * L_inv[i-1, j]
            
    U_inv = np.zeros((n, n), dtype=float)
    for i in range(n-1, -1, -1):
        U_inv[i, i] = 1.0 / U[i, i]
        for j in range(i+1, n):
            if i + 1 < n:
                U_inv[i, j] = - (U[i, i+1] * U_inv[i+1, j]) / U[i, i]
            else:
                U_inv[i, j] = 0.0
                
    A_inv = U_inv.dot(L_inv)
    return A_inv

def inverse_tridiagonal_thomas(A):
    n = A.shape[0]
    A_inv = np.zeros((n, n), dtype=float)
    I = np.eye(n)
    
    for i in range(n):
        A_inv[:, i] = solve_thomas(A, I[:, i])
    
    return A_inv

def solve_thomas(A, d):
    n = len(d)
    
    a = np.zeros(n)
    b = np.zeros(n)
    c = np.zeros(n)
    for i in range(n):
        b[i] = A[i][i]
        if i < n - 1:
            c[i] = A[i][i+1]
        if i > 0:
            a[i] = A[i][i-1]
    
    cp = np.zeros(n)
    dp = np.zeros(n)
    
    cp[0] = c[0] / b[0]
    dp[0] = d[0] / b[0]
    
    for i in range(1, n):
        denom = b[i] - a[i] * cp[i - 1]
        if i < n - 1:
            cp[i] = c[i] / denom
        dp[i] = (d[i] - a[i] * dp[i - 1]) / denom
    
    x = np.zeros(n)
    x[-1] = dp[-1]
    for i in range(n - 2, -1, -1):
        x[i] = dp[i] - cp[i] * x[i + 1]
    
    return x
        
def verify_inverse_tridiagonal(A, f):
    A_inv = f(A)
    print("A:\n", A)
    print("A_inv:\n", A_inv)
    print("A_inv @ A:\n", A_inv @ A)
    if np.allclose(A_inv @ A, np.eye(A.shape[0])):
        print("pass")
    else:
        print("fail")
        
verify_inverse_tridiagonal(build_tridiagonal(10, 1, -2, 1), inverse_tridiagonal_thomas)

A:
 [[-2  1  0  0  0  0  0  0  0  0]
 [ 1 -2  1  0  0  0  0  0  0  0]
 [ 0  1 -2  1  0  0  0  0  0  0]
 [ 0  0  1 -2  1  0  0  0  0  0]
 [ 0  0  0  1 -2  1  0  0  0  0]
 [ 0  0  0  0  1 -2  1  0  0  0]
 [ 0  0  0  0  0  1 -2  1  0  0]
 [ 0  0  0  0  0  0  1 -2  1  0]
 [ 0  0  0  0  0  0  0  1 -2  1]
 [ 0  0  0  0  0  0  0  0  1 -2]]
A_inv:
 [[-0.90909091 -0.81818182 -0.72727273 -0.63636364 -0.54545455 -0.45454545
  -0.36363636 -0.27272727 -0.18181818 -0.09090909]
 [-0.81818182 -1.63636364 -1.45454545 -1.27272727 -1.09090909 -0.90909091
  -0.72727273 -0.54545455 -0.36363636 -0.18181818]
 [-0.72727273 -1.45454545 -2.18181818 -1.90909091 -1.63636364 -1.36363636
  -1.09090909 -0.81818182 -0.54545455 -0.27272727]
 [-0.63636364 -1.27272727 -1.90909091 -2.54545455 -2.18181818 -1.81818182
  -1.45454545 -1.09090909 -0.72727273 -0.36363636]
 [-0.54545455 -1.09090909 -1.63636364 -2.18181818 -2.72727273 -2.27272727
  -1.81818182 -1.36363636 -0.90909091 -0.45454545]
 [-0.45454545 -0.90909091 -1.363